In [ ]:
from src import Color
from src.material.material.phong_material import PhongMaterial
from src import World
from src import Sphere, Plane
from src import PointLight
from src import ipynb_display_image
from src import Resolution
from src import Camera
from src import Vertex, Vector
from src import Scene
from src.render import LinearRayCaster
from src.render.loops.progress import PreviewConfig, ProgressDisplay
from src.render.render_config import RenderConfig

In [ ]:
# colors
brown = Color.custom_rgb(150, 75, 0)
red   = Color.custom_rgb(255, 0, 0)
white = Color.custom_rgb(245, 245, 245)
green = Color.custom_rgb(30, 180, 120)
gray  = Color.custom_rgb(200, 200, 200)

# --- material (vary transparency only) ---
glass_spec = Color.custom_rgb(255, 255, 255)  # specular for dielectric
# keep shininess/ior constant; reflectivity=0 to isolate transmission
transp_0  = PhongMaterial(name="transp_0",  base_color=red, spec_color=glass_spec, shininess=64, ior=1.5, reflectivity=0.0, transparency=0.0)
transp_25 = PhongMaterial(name="transp_25", base_color=red, spec_color=glass_spec, shininess=64, ior=1.5, reflectivity=0.0, transparency=0.25)
transp_50 = PhongMaterial(name="transp_50", base_color=red, spec_color=glass_spec, shininess=64, ior=1.5, reflectivity=0.0, transparency=0.50)
transp_90 = PhongMaterial(name="transp_90", base_color=red, spec_color=glass_spec, shininess=64, ior=1.5, reflectivity=0.0, transparency=0.90)
matte_ground = PhongMaterial(name="matte_ground", base_color=gray,
                             spec_color=Color.custom_rgb(80, 80, 80),
                             shininess=8, reflectivity=0.0)

# --- objects (left→right increases transparency) ---
sphere1 = Sphere(center=Vertex(-3.5, 0.2, -5), radius=1.0, material=transp_0)
sphere2 = Sphere(center=Vertex(-1.1, 0.2, -5), radius=1.0, material=transp_25)
sphere3 = Sphere(center=Vertex(1.1, 0.2, -5), radius=1.0, material=transp_50)
sphere4 = Sphere(center=Vertex(3.5, 0.2, -5), radius=1.0, material=transp_90)
plane = Plane(point=Vertex(0, -1, 0), normal=Vector(0, 1, 0), material=matte_ground)

# --- world ---
world = World()
world.add(sphere1)
world.add(sphere2)
world.add(sphere3)
world.add(sphere4)
world.add(plane)


# --- lights ---
point_light = PointLight(position=Vertex(5, 6, -1), intensity=1600.0, falloff=0.02)

# --- camera / scene ---
camera = Camera(
    fov=65,
    aspect_ratio=Resolution.R360p.aspect_ratio,
    origin=Vertex(0, 0, 0),
    direction=Vector(0, 0, -1),
)

scene = Scene(
    camera=camera,
    world=world,
    lights=[point_light],
    skybox_path ="../skybox/shanghai_4k.hdr"
)

scene.validate()

In [ ]:
render_configuration = RenderConfig(
    resolution=Resolution.R360p,
    samples_per_pixel=1,
    max_depth=4,
)

#todo fix TQDM_BAR not working because iam not updating the progress properly in the ray tracer
preview_configuration = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

my_ray_tracer = LinearRayCaster(scene=scene, render_config=render_configuration, preview_config=preview_configuration)
png = my_ray_tracer.render("images/shines.png")